In [7]:
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint#, TensorBoard
import pickle
from models import AlexNet, LeNetRegressor
from keras import backend as K
K.set_image_data_format('channels_last')

In [2]:
test_dataset = BatchGenerator(box_output_format=['class_id'])

In [3]:
test_dataset.parse_csv(labels_filename='/home/nicoli/github/ssd_keras/dataset/csv/imdb_csv/imdb_age_regression_test_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/ssd_keras/dataset/',
                        input_format=['image_name', 'class_id'])

In [4]:
img_height, img_width, img_depth = (224,224,3)

epochs = 90

train_batch_size = 64
shuffle = True
ssd_train = False

validation_batch_size = 32


# In[15]:

test_generator = test_dataset.generate(batch_size=train_batch_size,
                                       shuffle=shuffle,
                                       ssd_train=ssd_train,
                                       divide_by_stddev = 225,
                                       equalize=True,
                                       returns={'processed_labels'},
                                       resize=(img_height, img_width))

In [5]:
print("Number of images in the dataset:", test_dataset.get_n_samples())

Number of images in the dataset: 9590


In [6]:
steps = test_dataset.get_n_samples()/train_batch_size

In [8]:
alexnet = LeNetRegressor(1, img_width, img_height, img_depth, ativacao='relu', weights_path='callbacks/lenet/age/class-weights-reg-fase2-1.43-200.95.hdf5')


# In[7]:


alexnet.model.summary()
#alexnet.model.load_weights('callbacks/alexnet/age/weights.24-1658.03.hdf5')


/home/nicoli/github/alexnet/models.py:78: UserWarning: Considering a regression task with output function being 'relu'
  warnings.warn('Considering a regression task with output function being \'relu\'')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 220, 220, 6)       456       
_________________________________________________________________
activation_1 (Activation)    (None, 220, 220, 6)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 6)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 106, 106, 16)      2416      
_________________________________________________________________
activation_2 (Activation)    (None, 106, 106, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 44944)             0         
__________

In [9]:
optimizer = Adam(lr=0.0000001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000000001, amsgrad=True)

In [10]:
alexnet.model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [11]:
stats = alexnet.model.evaluate_generator(test_generator,steps=steps)

from math import sqrt
sqrt(stats[0]), stats

(17.926342486446185, [321.35375494136554, 14.091725622302425])